In [274]:
import pandas as pd
import datetime as dt
import numpy as np
import seaborn as sns
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
%config InlineBackend. figure_format = 'retina'

# Part 2 (一)銷售概況分析

In [16]:
new_all_data = pd.read_csv('new_all_data.csv')

In [17]:
all_M_profit = new_all_data[['Year', 'Month', '單價', '成本', '獲利']]
all_M_profit

,Year,Month,單價,成本,獲利
0,2016,2016-01,643.195,394.800,248.395
1,2016,2016-01,391.510,225.365,166.145
2,2016,2016-01,713.930,416.185,297.745
3,2016,2016-01,557.655,337.225,220.430
4,2016,2016-01,628.390,366.835,261.555
...,...,...,...,...,...
353219,2019,2019-11,575.750,403.025,172.725
353220,2019,2019-11,575.750,403.025,172.725
353221,2019,2019-11,575.750,403.025,172.725
353222,2019,2019-11,575.750,403.025,172.725


In [18]:
all_M_profit = all_M_profit.groupby(['Year', 'Month'], as_index=False)['單價', '成本', '獲利'].sum()

C:\Users\chanm\AppData\Local\Temp\ipykernel_18748\86225934.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  all_M_profit = all_M_profit.groupby(['Year', 'Month'], as_index=False)['單價', '成本', '獲利'].sum()


In [19]:
all_M_profit = all_M_profit.rename(columns={'單價':'營收'})

In [20]:
# 只取2018,2019年
profit_M_18_19 = all_M_profit[(all_M_profit['Year'] == 2018)|(all_M_profit['Year'] == 2019)]

In [21]:
# 刪除不適合比較的月份
profit_M_18_19 = profit_M_18_19.drop([34,35,46])

In [22]:
profit_M_18_19 = profit_M_18_19.reset_index(drop=True)

In [23]:
profit_Y_18_19 = profit_M_18_19.groupby(['Year'], as_index=False)['營收', '成本', '獲利'].sum()

C:\Users\chanm\AppData\Local\Temp\ipykernel_18748\1591178153.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  profit_Y_18_19 = profit_M_18_19.groupby(['Year'], as_index=False)['營收', '成本', '獲利'].sum()


In [24]:
profit_Y_18_19['成本佔比'] = round((profit_Y_18_19['成本']/profit_Y_18_19['營收'])*100, 1)

In [25]:
profit_Y_18_19['獲利佔比'] = round((profit_Y_18_19['獲利']/profit_Y_18_19['營收'])*100, 1)

In [27]:
profit_Y_18_19['Year'] = profit_Y_18_19['Year'].astype('str')

In [28]:
profit_Y_18_19 = profit_Y_18_19.T

In [29]:
profit_Y_18_19 = profit_Y_18_19.rename(columns={0:'2018年'})
profit_Y_18_19 = profit_Y_18_19.rename(columns={1:'2019年'})

In [30]:
profit_Y_18_19.drop('Year', axis = 0, inplace=True)

In [31]:
profit_Y_18_19 = profit_Y_18_19.astype(float)

In [32]:
profit_Y_18_19 = round(profit_Y_18_19)

In [33]:
profit_Y_18_19 = profit_Y_18_19.astype(int)

In [34]:
profit_Y_18_19['成長率'] = (profit_Y_18_19['2019年'] - profit_Y_18_19['2018年'])/profit_Y_18_19['2019年'] * 100
profit_Y_18_19['成長率'] = profit_Y_18_19['成長率'].astype(int)
profit_Y_18_19['成長率'] = round(profit_Y_18_19['成長率'], 1)


In [35]:
profit_Y_18_19 = profit_Y_18_19.rename(columns={'成長率':'成長率(%)'})

In [36]:
profit_Y_18_19.to_csv('profit_Y_18_19.csv', encoding='utf-8-sig')

In [37]:
profit_Y_18_19.loc['成本佔比':'獲利佔比','成長率(%)'] = np.nan 

In [38]:
profit_Y_18_19

,2018年,2019年,成長率(%)
營收,66794064,62321833,-7.0
成本,32612753,29465508,-10.0
獲利,34181310,32856325,-4.0
成本佔比,49,47,NaN
獲利佔比,51,53,NaN


In [39]:
profit_Y_18_19_fig = profit_Y_18_19.iloc[3:5, :2]
profit_Y_18_19_fig
fig_profit_Y_18_19 = px.pie(profit_Y_18_19_fig, values='2018年', names=['成本佔比','獲利佔比'], title='2018年度營收狀況', hole=.5)
fig_profit_Y_18_19.show()

In [40]:
fig_profit_Y_18_19 = px.pie(profit_Y_18_19_fig, values='2019年', names=['成本佔比',	'獲利佔比'], title='2019年度營收狀況', hole=.5)
fig_profit_Y_18_19.show()

In [41]:
all_M_profit.to_csv('all_M_profit.csv', encoding='utf-8-sig')
profit_M_18_19.to_csv('profit_M_18_19.csv', encoding='utf-8-sig')

# Part 2 (二)毛利率分析
產出2019年高GPM的data_2019銷售資料

In [17]:
import pandas as pd
import datetime as dt
import numpy as np
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [3]:
all_data = pd.read_csv('new_all_data.csv')

In [4]:
data_2019 = all_data[all_data['Year']==2019]

In [5]:
# 刪除兩筆售價為0的訂單(導致GPM = -inf)
# df.index[條件].tolist()
#all_data.index[all_data['單價'] == 0].tolist()
#all_data = all_data.drop(index = [285228, 313012])

In [6]:
high_GPM_2019 = (data_2019.groupby(['產品', '單價', '成本', '獲利', 'GPM'], as_index=False)['order_count'].count()).sort_values('order_count', ascending=False)

In [7]:
high_GPM_2019

,產品,單價,成本,獲利,GPM,order_count
980,產品13-12,985.355,450.730,534.625,54.3,638
2346,產品20-12,985.355,450.730,534.625,54.3,532
2344,產品20-12,985.355,432.635,552.720,56.1,529
1029,產品13-3,985.355,432.635,552.720,56.1,515
978,產品13-12,985.355,432.635,552.720,56.1,501
...,...,...,...,...,...,...
2845,產品26-11,1135.050,470.470,664.580,58.6,1
2847,產品26-12,820.855,401.380,419.475,51.1,1
2855,產品26-12,1061.025,383.285,677.740,63.9,1
2856,產品26-12,1061.025,401.380,659.645,62.2,1


In [8]:
# 'GPM_Rank'欄
high_GPM_2019 = high_GPM_2019.sort_values('GPM', ascending=False)
high_GPM_2019['GPM_Rank'] = range(1, len(high_GPM_2019)+1)

In [9]:

# 'Value'欄 = (price - cost) * count_order  | (price - cost) = GPM * price = profit
high_GPM_2019['Value'] = (high_GPM_2019['單價'] - high_GPM_2019['成本']) * high_GPM_2019['order_count']
high_GPM_2019['Value'] = high_GPM_2019['Value'].astype(int)


In [10]:
# 'value_rank'欄
high_GPM_2019 = high_GPM_2019.sort_values('Value', ascending=False)
high_GPM_2019['Value_Rank'] = range(1, len(high_GPM_2019)+1)


In [11]:
# 'Value_cunsum'欄
high_GPM_2019['Value_cunsum'] =high_GPM_2019['Value'].cumsum()

In [12]:
# '%_total_Value'欄
high_GPM_2019['%_total_Value'] = round(100*(high_GPM_2019['Value_cunsum']/data_2019['獲利'].sum()), 1)


In [13]:
high_GPM_2019 = high_GPM_2019[['產品', '單價', '成本', '獲利', 'order_count', 'GPM', 'GPM_Rank', 'Value', 'Value_Rank', 'Value_cunsum', '%_total_Value']]
high_GPM_2019.reset_index(drop=True, inplace=True)
high_GPM_2019

,產品,單價,成本,獲利,order_count,GPM,GPM_Rank,Value,Value_Rank,Value_cunsum,%_total_Value
0,產品13-12,985.355,450.730,534.625,638,54.3,3454,341090,1,341090,1.0
1,產品1-35,1628.550,689.255,939.295,345,57.7,1983,324056,2,665146,2.0
2,產品20-12,985.355,432.635,552.720,529,56.1,2579,292388,3,957534,2.9
3,產品13-3,985.355,432.635,552.720,515,56.1,2609,284650,4,1242184,3.7
4,產品20-12,985.355,450.730,534.625,532,54.3,3471,284420,5,1526604,4.6
...,...,...,...,...,...,...,...,...,...,...,...
6154,產品8-89,394.800,432.635,-37.835,17,-9.6,6154,-643,6155,33374971,100.0
6155,產品8-89,327.355,383.285,-55.930,12,-17.1,6159,-671,6156,33374300,100.0
6156,產品8-88,394.800,432.635,-37.835,22,-9.6,6153,-832,6157,33373468,100.0
6157,產品8-88,394.800,450.730,-55.930,20,-14.2,6158,-1118,6158,33372350,100.0


In [14]:
high_GPM_2019.to_csv('high_GPM_2019.csv')

In [ ]:
# GPM分布
# gpm = high_GPM_2019.groupby('GPM', as_index=False)['order_count','Value'].sum()
#gpm = gpm.sort_values('GPM', ascending=False)
#fig2 = px.bar(gpm, x="GPM", y="Value", color='Value')
#fig2.show()

# 以大於59.2(四分位數(75%))的GM篩選

In [44]:
np.percentile(high_GPM_2019['GPM'], (25, 50, 75))

array([47.2, 55.4, 59.2])

In [45]:
fig = px.box(data_2019['GPM'], x="GPM")
fig.show()
#sns.boxplot(x = data_2019['GPM'])

In [46]:

high_GPM_2019 = high_GPM_2019[high_GPM_2019['GPM'] > 59.2] #(np.percentile(high_GPM_2019['GPM'], (75)))]
high_GPM_2019 = high_GPM_2019.sort_values('GPM', ascending=False)

high_GPM_2019

,產品,單價,成本,獲利,order_count,GPM,GPM_Rank,Value,Value_Rank,Value_cunsum,%_total_Value
639,產品14-35,411.250,70.735,340.515,27,82.8,1,9193,640,24459147,73.3
1962,產品14-35,327.355,70.735,256.620,8,78.4,2,2052,1963,30047603,90.0
1845,產品43-35,491.855,116.795,375.060,6,76.3,3,2250,1846,29795073,89.3
680,產品45-174,1011.675,245.105,766.570,11,75.8,6,8432,681,24822880,74.4
531,產品45-177,1011.675,245.105,766.570,15,75.8,5,11498,532,23357647,70.0
...,...,...,...,...,...,...,...,...,...,...,...
1810,產品11-3,985.355,401.380,583.975,4,59.3,1398,2335,1811,29714935,89.0
1809,產品5-12,985.355,401.380,583.975,4,59.3,1391,2335,1810,29712600,89.0
2812,產品16-3,985.355,401.380,583.975,2,59.3,1327,1167,2813,31349692,93.9
506,產品127-12,985.355,401.380,583.975,21,59.3,1351,12263,507,23060342,69.1


In [444]:
print('2019年GPM大於59.2的訂單數佔2019年訂單數'+str(round((high_GPM_2019['order_count'].sum()) / len(data_2019) *100, 1)) + '%')


佔2019年訂單數22.1%


# 找出行銷策略優先採納的高GPM商品

In [445]:
# 從GPM前25%的商品，進一步篩選推薦清單: high_GPM_2019['Value'] > 50000 的產品
recommend_list =  high_GPM_2019[(high_GPM_2019['Value'] > 50000)] 

In [446]:
recommend_list = recommend_list.sort_values('Value', ascending=False)

In [447]:
# 加工:重新計算累績價值、以及佔所有產品價值的幾％、RESET_INDEX
recommend_list['Value_cunsum'] = recommend_list['Value'].cumsum()

In [448]:
recommend_list['%_total_Value'] = round(100*(recommend_list['Value_cunsum']/data_2019['獲利'].sum()), 1)

In [449]:
recommend_list.reset_index(drop= True, inplace = True)


In [450]:
print(len(recommend_list['產品'].tolist()))
recommend_list
# 以提高2019年GPM為目標的推薦產品清單，佔累積獲利約11.9%的商品組合，共有42項商品

42


,產品,單價,成本,獲利,order_count,GPM,GPM_Rank,Value,Value_Rank,Value_cunsum,%_total_Value
0,產品14-135,1135.050,378.350,756.700,304,66.7,109,230036,11,230036,0.7
1,產品3-35,1135.050,406.315,728.735,276,64.2,247,201130,12,431166,1.3
2,產品1-66,1135.050,430.990,704.060,264,62.0,574,185871,14,617037,1.8
3,產品14-132,970.550,383.285,587.265,280,60.5,1160,164434,17,781471,2.3
4,產品14-136,1299.550,378.350,921.200,174,70.9,21,160288,19,941759,2.8
5,產品13-2,477.050,139.825,337.225,442,70.7,69,149053,22,1090812,3.3
6,產品1-35,1135.050,379.995,755.055,192,66.5,112,144970,24,1235782,3.7
7,產品20-12,1135.050,432.635,702.415,196,61.9,663,137673,28,1373455,4.1
8,產品14-35,1135.050,411.250,723.800,183,63.8,437,132455,32,1505910,4.5
9,產品1-12,985.355,383.285,602.070,196,61.1,1121,118005,39,1623915,4.9
